In [166]:
from utils import find_needed_for,rows,find_by_name
import random
from heapq import heappush, heappop
import json

In [167]:
def print_storage_not_nan(storage):
    print("=====Storage=====")
    for k,v in storage.items():
        if v != 0:
            print(f"{k}: {v}")
    print("=================")

In [168]:
storage = {}
for row in rows:
    storage[row['Name']] = 0

storage['Peony'] = 2
storage['Cotton'] = 2
print_storage_not_nan(storage)

=====Storage=====
Cotton: 2
Peony: 2


In [169]:
sources = {
    'Field':[
        {
            'status':'idle',
            'product':None,
            'start_time':None,
            'end_time':None,
        },
        {
            'status':'idle',
            'product':None,
            'start_time':None,
            'end_time':None,
        },
    ],
    'Flower Shop':[
        {
            'status':'idle',
            'product':None,
            'start_time':None,
            'end_time':None,
        }
    ],
    'Loom':[
        {
            'status':'idle',
            'product':None,
            'start_time':None,
            'end_time':None,
        }
    ]
}

In [170]:
tasks = []
test = find_needed_for('Soft bouquet')

def add_to_queue(product):
    for need in product['Needs']:
        add_to_queue(need)
    for _ in range(product['Count']):
        tasks.append(product['Name'])
add_to_queue(test)

In [171]:
time_queue = [0]

print(tasks)
while len(time_queue) > 0:
    time = heappop(time_queue)
    print('Time:',time)
    for source_name in sources:
        for i,s in enumerate(sources[source_name]):
            if s['end_time'] == time:
                if source_name=='Field':
                    storage[s['product']] += 2
                else:
                    storage[s['product']] += 1
                s['status'] = 'idle'
                s['product'] = None
                s['start_time'] = None
                s['end_time'] = None
    print_storage_not_nan(storage)
    for i,task in enumerate(tasks.copy()):
        task = find_by_name(task)

        if_have_enough_materials = True
        for need in task['Needs']:
            key,value = list(need.items())[0]
            if storage[key] < value:
                if_have_enough_materials = False
                break
        print(task['Name'],i,'if_have_enough_materials',if_have_enough_materials)
        if not if_have_enough_materials:
            continue
        if_have_enough_sources = False
        source = task['Source']
        for s in sources[source]:
            if s['status'] == 'idle':
                if_have_enough_sources = True
                break
        print(task['Name'],i,'if_have_enough_sources',if_have_enough_sources)
        if not if_have_enough_sources:
            continue
        print('Schedule:',task['Name'])
        s['status'] = 'busy'
        s['product'] = task['Name']
        s['start_time'] = time
        s['end_time'] = time + task['Time']
        for need in task['Needs']:
            key,value = list(need.items())[0]
            storage[key] -= value
        if s['end_time'] not in time_queue:
            heappush(time_queue,s['end_time'])
        tasks.remove(task['Name'])
    print_storage_not_nan(storage)

['Peony', 'Peony', 'Peony', 'Peony', 'Cotton', 'Cotton', 'Cotton', 'Cotton', 'Cotton fabric', 'Soft bouquet']
Time: 0
=====Storage=====
Cotton: 2
Peony: 2
Peony 0 if_have_enough_materials True
Peony 0 if_have_enough_sources True
Schedule: Peony
Peony 1 if_have_enough_materials True
Peony 1 if_have_enough_sources True
Schedule: Peony
Peony 2 if_have_enough_materials False
Peony 3 if_have_enough_materials False
Cotton 4 if_have_enough_materials True
Cotton 4 if_have_enough_sources False
Cotton 5 if_have_enough_materials True
Cotton 5 if_have_enough_sources False
Cotton 6 if_have_enough_materials True
Cotton 6 if_have_enough_sources False
Cotton 7 if_have_enough_materials True
Cotton 7 if_have_enough_sources False
Cotton fabric 8 if_have_enough_materials False
Soft bouquet 9 if_have_enough_materials False
=====Storage=====
Cotton: 2
Time: 14400
=====Storage=====
Cotton: 2
Peony: 4
Peony 0 if_have_enough_materials True
Peony 0 if_have_enough_sources True
Schedule: Peony
Peony 1 if_have_eno

In [172]:
print_storage_not_nan(storage)


=====Storage=====
Cotton: 2
Peony: 2
Soft bouquet: 1
